# 1. 데이터 전처리. 정제가 잘 된 데이터이기 때문에 딱히 손 볼 곳이 없다.

In [2]:
import pandas as pd

# 훈련/테스트 데이터를 읽어온다.
train = pd.read_csv('./train.csv')
train_label = train['target']  # 타겟과
train_id = train['id']   # 아이디 분리하기
del train['target'], train['id']
# train 셋의 train label(target), train id 구분지은 후 원 값 삭제


test = pd.read_csv('./test.csv')
test_id = test['id']
del test['id']
# test 셋의 test id 로 변수 설정 후 원 값 삭제

# 2. 피처 엔지니어링

### 1) 결측값의 개수를 나타내는 missing 변수
### 2) 이진 변수들의 총합
### 3) Target Encoding 파생 변수

In [4]:
# 결측값을 의미하는 missing 변수인 -1의 개수를 샌다. (파생변수1이라 명명)
train['missing'] = (train==-1).sum(axis=1).astype(float)
test['missing'] = (test==-1).sum(axis=1).astype(float)


# 결측값은 필자의 생각으로는 새로운 군집 정보를 제공할 수 있다고 생각한다. 예를 들어, 갓 운전을 시작하여 포르토 세구로 계정을 생성한 경우 숙련된 운전자의 데이터에 비해 결측값이 더 많이 존재할 수 있다.
# 숙련된 운전자는 오랜 기간동안 포르토 세구로 계정을 사용하며 많은 정보가 쌓인 반면, 초보 운전자들의 정보는 정보가 적을 수 있기 때문이다.

In [10]:
sum(train['missing'])

846458.0

In [11]:
# 이진 변수의 합을 센다 (파생변수2이라 명명) 
bin_features = [c for c in train.columns if 'bin' in c]  # 피처명 중 bin 이 들어간 것들을 센다.
train['bin_sum'] = train[bin_features].sum(axis=1)
test['bin_sum'] = test[bin_features].sum(axis=1)

In [13]:
print(bin_features)

['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_10_bin', 'ps_ind_11_bin', 'ps_ind_12_bin', 'ps_ind_13_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin', 'ps_calc_15_bin', 'ps_calc_16_bin', 'ps_calc_17_bin', 'ps_calc_18_bin', 'ps_calc_19_bin', 'ps_calc_20_bin']


In [16]:
sum(train['bin_sum'])

2405713

In [18]:
# 타겟 인코딩을 수행하기 위한 단일변수 타겟 비율 분석 (파생변수3 이라 명명)  -> 주로 범주형 변수에서 좋은 성능을 보인다.
## 타겟 엔지니어링은 단일 변수의 고유값별 타겟 변수의 평균값을 파생 변수로 활용하는 피처 엔지니어링 기법이다.
## 예를들어, 운전자 A의 'ps_ind_01'의 변수 값이 0일 경우, 'ps_ind_01' 변수 값이 0인 모든 운전자들의 평균 타겟 값을 'ps_ind_01_target_enc' 파생 변수로 사용하는 것이다.

### 타겟 변수의 값을 직접적으로 사용하는 변수이기에, 구현을 잘못할 경우 데이터 유출로 이어져 모델 파이프라인이 망가질 수 있다.
### 데이터 유출을 방지하기 위형, 5-Fold 내부 교차ㅣ 검증 과정에서 학습에 사용되는 4/5의 훈련 데이터로 변수 고유값 별 평균 타겟값을 계산
#  -> 1/5의 검증 데이터에 해당 값을 매핑하는 방식을 취한다.


features = ['ps_ind_06_bin', 'ps_ind_07_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin', 'ps_ind_16_bin', 'ps_ind_17_bin', 'ps_ind_18_bin',
           'ps_ind_04_cat', 'ps_ind_05_cat', 'ps_car_01_cat','ps_car_02_cat','ps_car_03_cat','ps_car_04_cat','ps_car_06_cat','ps_car_07_cat','ps_car_08_cat','ps_car_09_cat', 'ps_car_11_cat',
           'ps_ind_01','ps_ind_03','ps_ind_15','ps_car_11']